In [1]:
import osmnx as ox
import networkx as nx
import pandas as pd


In [2]:
# Unduh graf jalan untuk wilayah Surabaya
graph = ox.graph_from_place("Surabaya, Indonesia", network_type="drive")

# Fungsi untuk menghitung estimasi waktu perjalanan
def estimate_travel_time(distance_km, traffic_condition, vehicle_type):
    # Kecepatan rata-rata berdasarkan kondisi lalu lintas (km/jam)
    speeds = {
        "lancar": {"mobil": 50, "motor": 60},
        "sedang": {"mobil": 30, "motor": 40},
        "macet": {"mobil": 15, "motor": 20},
    }
    
    # Dapatkan kecepatan untuk jenis kendaraan dan kondisi lalu lintas
    speed = speeds.get(traffic_condition, {}).get(vehicle_type, 30)  # Default: 30 km/jam
    travel_time = (distance_km / speed) * 60  # Konversi ke menit
    return travel_time

In [9]:
# Fungsi untuk mendapatkan jarak dan rute menggunakan OSMnx
def get_osm_route_with_time(graph, start_lat, start_lon, end_lat, end_lon, traffic_condition, vehicle_type):
    try:
        # Temukan node terdekat
        start_node = ox.distance.nearest_nodes(graph, start_lon, start_lat)
        end_node = ox.distance.nearest_nodes(graph, end_lon, end_lat)
        
        # Temukan rute terpendek berdasarkan panjang jalan
        route = nx.shortest_path(graph, source=start_node, target=end_node, weight="length")
        
        # Hitung jarak total
        distance = sum(ox.utils_graph.get_route_edge_attributes(graph, route, "length")) / 1000  # Dalam km
        
        # Estimasi waktu perjalanan
        travel_time = estimate_travel_time(distance, traffic_condition, vehicle_type)
        
        return route, distance, travel_time
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None

In [10]:
# Daftar lokasi di Surabaya
locations = [
    {"name": "Stasiun Gubeng", "lat": -7.257472, "lon": 112.752088},
    {"name": "Tunjungan Plaza", "lat": -7.266735, "lon": 112.736115},
    {"name": "Kebun Binatang Surabaya", "lat": -7.290215, "lon": 112.735337},
    {"name": "Surabaya Town Square", "lat": -7.290116, "lon": 112.734558},
    {"name": "Grand City Mall", "lat": -7.261414, "lon": 112.751639},
    {"name": "Terminal Bungurasih", "lat": -7.345473, "lon": 112.742177},
    {"name": "Pelabuhan Tanjung Perak", "lat": -7.203854, "lon": 112.723741},
    {"name": "Galaxy Mall", "lat": -7.273925, "lon": 112.779602},
    {"name": "Pakuwon Mall", "lat": -7.291418, "lon": 112.677844},
    {"name": "Stadion Gelora Bung Tomo", "lat": -7.268027, "lon": 112.630660},
]


In [11]:
# Generate data untuk setiap pasangan lokasi
results = []
traffic_conditions = ["lancar", "sedang", "macet"]  # Kondisi lalu lintas

for i, start in enumerate(locations):
    for j, end in enumerate(locations):
        if i != j:  # Hindari rute ke lokasi yang sama
            for condition in traffic_conditions:
                # Hitung untuk mobil
                _, distance, travel_time_mobil = get_osm_route_with_time(
                    graph, start["lat"], start["lon"], end["lat"], end["lon"], condition, "mobil"
                )
                
                # Hitung untuk motor
                _, _, travel_time_motor = get_osm_route_with_time(
                    graph, start["lat"], start["lon"], end["lat"], end["lon"], condition, "motor"
                )
                
                if distance and travel_time_mobil and travel_time_motor:
                    results.append([
                        start["name"], start["lat"], start["lon"],
                        end["name"], end["lat"], end["lon"],
                        condition,  # Kondisi lalu lintas
                        distance,   # Jarak (km)
                        travel_time_mobil,  # Waktu (mobil)
                        travel_time_motor   # Waktu (motor)
                    ])


/var/folders/bd/1gfn4wvd2yj_rjzsdjyq1g940000gn/T/ipykernel_66324/335626722.py:12: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  distance = sum(ox.utils_graph.get_route_edge_attributes(graph, route, "length")) / 1000  # Dalam km
/var/folders/bd/1gfn4wvd2yj_rjzsdjyq1g940000gn/T/ipykernel_66324/335626722.py:12: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  distance = sum(ox.utils_graph.get_route_edge_attributes(graph, route, "length")) / 1000  # Dalam km
/var/folders/bd/1gfn4wvd2yj_rjzsdjyq1g940000gn/T/ipykernel_66324/335626722.py:12: FutureWarning: The `get_route_edge_attributes` function 

In [ ]:
# Simpan hasil ke file CSV
df = pd.DataFrame(results, columns=[
    "Start_Name", "Start_Lat", "Start_Lon",
    "End_Name", "End_Lat", "End_Lon",
    "Traffic_Condition", "Distance_km", "Travel_Time_Mobil_min", "Travel_Time_Motor_min"
])
df.to_csv("surabaya_routes_with_vehicle_times.csv", index=False)
print("Data berhasil disimpan ke surabaya_routes_with_vehicle_times.csv")

Data berhasil disimpan ke surabaya_routes_with_vehicle_times.csv
